# 🔄 Weekly Refresh - Add New Games to Rolling Dataset

**Run this every Sunday to stay current!**

**What this does:**
1. Fetches last 7 days of 2025-26 games
2. Adds them to your rolling dataset
3. Removes any games older than 5 years
4. Updates metadata

**Takes:** ~5 minutes  
**Frequency:** Weekly (or after big game nights)

---

## Step 1: Load Current Rolling Dataset

In [ ]:
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
import json

# Load your rolling dataset
DATA_DIR = Path(r"C:\Users\wdors\qepc_project\experimental\CLAUDE_REWRITE\data\raw\player_logs")
rolling_file = DATA_DIR / "player_logs_rolling_5yr.csv"

print(f"📂 Loading current dataset...")
print(f"   From: {rolling_file}")

df_current = pd.read_csv(rolling_file)
df_current['GAME_DATE'] = pd.to_datetime(df_current['GAME_DATE'])

latest_game = df_current['GAME_DATE'].max()

print(f"\n✅ Current dataset:")
print(f"   Records: {len(df_current):,}")
print(f"   Latest game: {latest_game.date()}")
print(f"   Players: {df_current['PLAYER_NAME'].nunique():,}")
print()

# How many days behind are we?
days_behind = (datetime.now() - latest_game).days
print(f"⏰ Status:")
if days_behind <= 2:
    print(f"   ✅ Up to date! (Only {days_behind} days behind)")
elif days_behind <= 7:
    print(f"   ⚠️  {days_behind} days behind - time to refresh")
else:
    print(f"   🔴 {days_behind} days behind - definitely need to refresh!")

---

## Step 2: Fetch New Games (Last 7 Days)

In [ ]:
from nba_api.stats.endpoints import leaguegamelog
import time

print(f"🏀 Fetching new games from 2025-26 season...")
print(f"   Season: 2025-26")
print(f"   Will get ALL games, then filter to new ones")
print()

try:
    # Fetch ALL 2025-26 games
    print("   Calling NBA API...")
    season_data = leaguegamelog.LeagueGameLog(
        season='2025',
        season_type_all_star='Regular Season',
        player_or_team_abbreviation='P'
    )
    
    df_all_2025 = season_data.get_data_frames()[0]
    df_all_2025['GAME_DATE'] = pd.to_datetime(df_all_2025['GAME_DATE'])
    
    # Filter to only NEW games (after our latest game)
    df_new = df_all_2025[df_all_2025['GAME_DATE'] > latest_game].copy()
    
    if len(df_new) > 0:
        print(f"\n✅ Found NEW games:")
        print(f"   New records: {len(df_new):,}")
        print(f"   Date range: {df_new['GAME_DATE'].min().date()} to {df_new['GAME_DATE'].max().date()}")
        print(f"   Players: {df_new['PLAYER_NAME'].nunique():,}")
        
        # Add season year
        df_new['SEASON_YEAR'] = '2025-26'
        
        # Show sample
        print(f"\n📋 Recent games:")
        sample = df_new.nlargest(5, 'GAME_DATE')[['GAME_DATE', 'PLAYER_NAME', 'PTS', 'REB', 'AST']]
        print(sample.to_string(index=False))
    else:
        print(f"\n✅ No new games found - you're already up to date!")
        df_new = pd.DataFrame(columns=df_current.columns)
    
except Exception as e:
    print(f"\n❌ Error fetching from NBA API: {e}")
    print(f"\n💡 Possible reasons:")
    print(f"   - Rate limit (wait a few minutes and try again)")
    print(f"   - API authentication issue")
    print(f"   - Network connectivity")
    
    df_new = pd.DataFrame(columns=df_current.columns)
    print(f"\n⏭️  Skipping this refresh - try again later")

---

## Step 3: Merge New Games

In [ ]:
if len(df_new) > 0:
    print(f"🔗 Merging new games into rolling dataset...")
    print()
    
    # Combine
    df_updated = pd.concat([df_current, df_new], ignore_index=True)
    
    # Remove duplicates (just in case)
    before_dedup = len(df_updated)
    df_updated = df_updated.drop_duplicates(subset=['PLAYER_ID', 'GAME_ID'], keep='last')
    duplicates_removed = before_dedup - len(df_updated)
    
    if duplicates_removed > 0:
        print(f"   🗑️  Removed {duplicates_removed} duplicate records")
    
    # Remove games older than 5 years (rolling window)
    cutoff_date = pd.Timestamp('2021-10-01')  # Start of 2021-22 season
    before_trim = len(df_updated)
    df_updated = df_updated[df_updated['GAME_DATE'] >= cutoff_date]
    old_games_removed = before_trim - len(df_updated)
    
    if old_games_removed > 0:
        print(f"   🗑️  Removed {old_games_removed} games older than Oct 2021")
    
    # Sort
    df_updated = df_updated.sort_values(['PLAYER_ID', 'GAME_DATE']).reset_index(drop=True)
    
    print(f"\n✅ Updated dataset:")
    print(f"   Total records: {len(df_updated):,} (was {len(df_current):,})")
    print(f"   Net change: +{len(df_updated) - len(df_current):,} records")
    print(f"   Latest game: {df_updated['GAME_DATE'].max().date()}")
    
else:
    print(f"⏭️  No new games to merge")
    df_updated = df_current

---

## Step 4: Save Updated Dataset

In [ ]:
if len(df_new) > 0:
    print(f"💾 Saving updated rolling dataset...")
    
    # Save
    df_updated.to_csv(rolling_file, index=False)
    
    file_size_mb = rolling_file.stat().st_size / 1024 / 1024
    
    print(f"\n✅ Saved successfully!")
    print(f"   Location: {rolling_file}")
    print(f"   Size: {file_size_mb:.1f} MB")
    print(f"   Records: {len(df_updated):,}")
    
    # Update metadata
    metadata = {
        'last_updated': datetime.now().isoformat(),
        'last_updated_date': datetime.now().strftime('%B %d, %Y'),
        'records': len(df_updated),
        'players': int(df_updated['PLAYER_NAME'].nunique()),
        'date_range': f"{df_updated['GAME_DATE'].min().date()} to {df_updated['GAME_DATE'].max().date()}",
        'seasons': '2021-22, 2022-23, 2023-24, 2024-25, 2025-26 (partial)',
        'file_size_mb': float(f"{file_size_mb:.1f}"),
        'new_games_added': len(df_new),
        'data_window': '5 year rolling window'
    }
    
    metadata_file = DATA_DIR / "player_logs_metadata.json"
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"   Metadata: Updated")
    print(f"\n🎉 Refresh complete!")
    
else:
    print(f"✅ Already up to date - no save needed")

---

## Summary

In [ ]:
print("="*60)
print("📊 REFRESH SUMMARY")
print("="*60)
print()

if len(df_new) > 0:
    print(f"✅ SUCCESS!")
    print()
    print(f"Added to dataset:")
    print(f"   New games: {len(df_new):,}")
    print(f"   Date range: {df_new['GAME_DATE'].min().date()} to {df_new['GAME_DATE'].max().date()}")
    print()
    print(f"Your rolling dataset now has:")
    print(f"   Total records: {len(df_updated):,}")
    print(f"   Through: {df_updated['GAME_DATE'].max().date()}")
    print()
    print(f"🔄 Next refresh: Run this again next week!")
else:
    print(f"✅ Already up to date!")
    print()
    print(f"Your dataset:")
    print(f"   Records: {len(df_current):,}")
    print(f"   Through: {df_current['GAME_DATE'].max().date()}")
    print()
    print(f"🔄 Check back in a few days for new games")

print()
print("="*60)

---

## ✅ Refresh Complete!

**Your rolling dataset stays current with:**
- ✅ Last 4 complete seasons (2021-22 through 2024-25)
- ✅ Current season (2025-26) through latest games
- ✅ Automatic 5-year window (drops old games)

**Run this notebook:**
- 📅 Weekly (every Sunday)
- 🎯 After big game nights
- 🔄 Anytime you want fresh data

**Your quantum props system now has the freshest data!** 🚀